# Segmentation Workflow

## Install packages

In [4]:
print("Installing packages (this could take awhile)...")

import sys
!$sys.executable -m pip install  -r ../requirements.txt --quiet --exists-action i
print("Done!")

Installing packages (this could take awhile)...
Done!


In [3]:
import subprocess
import os
from pathlib import Path
import pathlib
from typing import Optional, Dict
import re
import numpy as np
import time
import warnings
from bfio import BioReader, BioWriter
import skimage.measure as measure
import multiprocessing
from functools import partial
module_path = os.path.abspath(os.path.join('../utils'))
if module_path not in sys.path:
    sys.path.append(module_path)
from func import *

## Inputs

In [5]:
workDir = pathlib.Path(os.getcwd()).parent
inpDir = pathlib.Path('/opt/shared/notebooks/eastman/fulldata/polus-data/polus/images/Eastman2021Infectivity/standard/intensity')
platesNum=200
modelDir = pathlib.Path('/home/jovyan/shared/notebooks/eastman/model/UnetPlusPlus-MCCLoss-efficientnet-b6-imagenet-Adam')
outDir = pathlib.Path('/opt/shared/notebooks/eastman/tmp5')


In [14]:
logging.basicConfig(
    format='%(asctime)s - %(name)-8s - %(levelname)-8s - %(message)s',
    datefmt='%d-%b-%y %H:%M:%S',
)
logger = logging.getLogger('Segmentation Workflow')
logger.setLevel(logging.INFO)

warnings.filterwarnings('ignore')

@my_timer
def segmentation_plugins_workflow_1(inpDir:pathlib.Path,
                           outDir:pathlib.Path,
                           modelDir:pathlib.Path,
                           platesNum:int,
                           dryrun:bool=True):
    
        os.environ["CUDA_VISIBLE_DEVICES"] = "0"
        starttime= time.time() 

        plates = [str(i).zfill(3) for i in range(1, platesNum+1)]

        assert platesNum != 0, "Number of Processing Plates should be greater than zero"

        filePatterns = ['p'+ pl +  '_x{x+}_y{y+}_wx{t}_wy{p}_c2.ome.tif'  for pl in plates]

        assert len(filePatterns) == platesNum, "Number of filepattern should be equivalent to number of plates"


        for plate, filePattern in zip(plates, filePatterns):


            logger.info(f'Segmentation/Analysis workflow starts for plate:{plate} & filePattern:{filePattern}')

            logger.info(f'Running 1st plugin: basic_flatfield on plate: p{plate}')

            bpath = basic_flatfield(inpDir,
                                       outDir,
                                       filePattern,
                                       plate,
                                       dryrun=dryrun)


            logger.info(f'Running 2nd plugin: apply_flatfield on plate: p{plate}')

            corrDir = apply_flatfield(inpDir,
                               outDir,
                               ffDir=bpath,
                               filePattern=filePattern,
                               plate=plate,
                               dryrun=dryrun)

            logger.info(f'Running 3rd plugin: SMP training/inference on plate: p{plate}')

            outpath = SMP_training_inference(inpDir=corrDir, 
                               modelDir=modelDir, 
                               filePattern=filePattern,
                               plate=plate,
                               outDir=outDir,
                               dryrun=dryrun)
            
            
            
        return corrDir, outpath
    
    
@my_timer
def feature_extraction_workflow(inpDir:pathlib.Path,
                           outDir:pathlib.Path,
                           corrDir:pathlib.Path,
                           platesNum:int,
                           dryrun:bool=True):
    
    plate = [str(i).zfill(3) for i in range(1, platesNum+1)][:2]

    assert platesNum != 0, "Number of Processing Plates should be greater than zero"
    
    
    logger.info(f'Running 4th plugin:  labelling_images')


    labelpath = run_labelling_images(inpDir=inpDir,
                                   outDir=outDir,
                                   dryrun=dryrun)

    logger.info(f'Running 5th plugin:  Renaming image Files for channel')


    renamepath = rename_files(inpDir=labelpath,
                   dryrun=dryrun)

    logger.info(f'Running 6th plugin:  Nyxus Feature Extraction')


    run_nyxus_exe(inpDir=corrDir,
                  segDir=renamepath,
                  outDir=outDir,
                  plate=plate,
                  dryrun=dryrun
             )
    
    logger.info(f'Running 7th plugin:  Renaming of Nyxus CSVs & deleting subfolders')
    
    outnyxus = pathlib.Path(outDir, 'nyxus')
    
    outpath = renameCSVs(inpDir=outnyxus,
                         dryrun=dryrun
                        )
    
    logger.info(f'Running 8th plugin:  Merging CSVs')
    
    outpath = csv_merger(inpDir=outpath,
              outDir=outDir,
              dim="rows",
              dryrun=dryrun
              )
    
    return

    
        
corrDir, outpath = segmentation_plugins_workflow_1(inpDir=inpDir,
                           outDir=outDir,
                           modelDir=modelDir,
                           platesNum=platesNum,
                           dryrun=False                          
                          )  

featpath = feature_extraction_workflow(inpDir=outpath,
                           outDir=outDir,
                           corrDir=corrDir,
                           platesNum=platesNum,
                           dryrun=False)   
        
        
